In [ ]:
# Using:
# accelerate
# torch
# open-clip-torch
# transformers==4.50.3


In [16]:
from transformers import AutoModel, AutoProcessor
model = AutoModel.from_pretrained('Marqo/marqo-fashionSigLIP', trust_remote_code=True)
processor = AutoProcessor.from_pretrained('Marqo/marqo-fashionSigLIP', trust_remote_code=True)

You are using a model of type siglip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [13]:
import torch
from PIL import Image

image = [Image.open("/Users/somogyijanos/Repos/somogyijanos/smartrobe/test-images/00/2025_08_2009_01_320011.JPG")]
# image = [Image.open("/Users/somogyijanos/Repos/somogyijanos/smartrobe/test-images/10/IMG_0214.JPG")]
text = [
    "hat", "cap", "beanie", "t-shirt", "shirt", "blouse", "tank top",
    "polo shirt", "sweater", "hoodie", "cardigan", "jacket", "coat",
    "blazer", "vest", "jeans", "pants", "trousers", "shorts", "leggings", "sweatpants",
    "joggers", "chinos", "skirt", "dress", "jumpsuit", "romper", "shoes",
    "sneakers", "boots", "sandals", "heels", "flats", "loafers", "slippers", "athletic shoes",
    "dress shoes", "socks", "underwear", "bra", "swimsuit", "bikini", "swim trunks",
    "scarf", "gloves", "mittens", "belt", "tie", "bow tie", "jewelry", "necklace",
    "earrings", "bracelet", "watch", "ring", "sunglasses", "purse", "handbag", "backpack", "wallet"
]
processed = processor(text=text, images=image, padding='max_length', return_tensors="pt")

with torch.no_grad():
    image_features = model.get_image_features(processed['pixel_values'], normalize=True)
    text_features = model.get_text_features(processed['input_ids'], normalize=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)

# Get the top 3 most probable categories
top3_indices = text_probs.argsort(dim=-1, descending=True)[0][:3]
top3_categories = [text[idx] for idx in top3_indices]
top3_scores = [text_probs[0, idx].item() for idx in top3_indices]

print("Top 3 most probable categories:")
for i, (category, score) in enumerate(zip(top3_categories, top3_scores), 1):
    print(f"{i}. {category} (score: {score:.4f})")

Label probs: tensor([[3.6853e-05, 1.9796e-05, 2.6209e-06, 9.6477e-06, 5.2416e-04, 4.5374e-06,
         3.2650e-06, 2.9837e-05, 3.1002e-05, 6.9482e-07, 2.6150e-05, 2.3374e-04,
         2.5366e-04, 3.7345e-04, 8.5314e-05, 1.7657e-03, 1.9929e-01, 4.2040e-01,
         9.9107e-04, 2.4575e-04, 1.1863e-04, 7.2447e-04, 3.7191e-01, 2.2294e-05,
         9.7894e-05, 1.3273e-04, 1.5372e-05, 1.3254e-04, 2.6753e-05, 2.7658e-05,
         1.4968e-05, 8.0561e-06, 1.9771e-04, 9.3901e-04, 4.5827e-06, 2.5403e-06,
         5.5583e-04, 1.3674e-05, 1.1177e-04, 7.5405e-07, 7.2951e-06, 1.2556e-06,
         5.4257e-05, 1.1594e-06, 1.1010e-05, 2.8758e-06, 2.8536e-04, 9.9389e-05,
         3.5251e-05, 2.2810e-06, 8.9891e-08, 1.4575e-06, 3.9779e-06, 4.3518e-05,
         2.9554e-06, 4.0098e-06, 4.0959e-06, 2.7068e-06, 4.7752e-07, 5.9204e-05]])
Top 3 most probable categories:
1. trousers (score: 0.4204)
2. chinos (score: 0.3719)
3. pants (score: 0.1993)
